In [102]:
import numpy as np
import random

In [103]:
suits = {'Hearts', 'Clubs', 'Spades', 'Diamonds'}

card_name_map = {
            'Two': 2,
            'Three': 3,
            'Four': 4,
            'Five': 5,
            'Six': 6,
            'Seven': 7,
            'Eight': 8,
            'Nine': 9,
            'Ten': 10,
            'Jack': 11, 
            'Queen': 12, 
            'King': 13, 
            'Ace': 14
            }

In [104]:
class Card:
    def __init__(self, name, suit):
        self.suit = suit
        self.name = name
        self.value = card_name_map[self.name]
    
    def __str__(self):
        return '{} of {}'.format(self.name, self.suit)

In [119]:
class Deck:
    def __init__(self):
        self.deck = []
        for suit in suits:
            for card_name in card_name_map:
                self.deck.append(Card(card_name, suit))
        self.shuffle()
                
    def __str__(self):
        return ''.join(['{} \n'.format(card) for card in self.deck])
    
    def shuffle(self):
        random.shuffle(self.deck)
    
    def draw(self):
        return self.deck.pop(0)
        

In [114]:
class Player:
    def __init__(self, name, chips):
        self.chips = chips
        self.name = name
        self.hand = []
        self.pre_bet_moves = {'check', 'bet'}
        self.post_bet_moves = {'fold', 'call', 'raise'}
        
    def __str__(self):
        return '{}{}'.format('Player: {}\nChips: {}\n'.format(self.name, self.chips), ''.join(['{} \n'.format(card) for card in self.hand]))
    
    def bet(self, amount):
        self.chips. -= amount
        return amount
    
    
    

In [115]:
class Game:
    def __init__(self, players, buy_in):
        self.players = [Player('{}'.format(i), buy_in) for i in range(players)]
        self.deck = Deck()
        self.pot = 0
        
        
    def __str__(self):
        return ''.join(['{}\n'.format(player) for player in self.players])
        
    def deal(self):
        [[player.hand.append(card) for card in [self.deck.draw() for i in range(2)]] for player in self.players]
        
    def flop(self):
        return ''.join(['{}\n'.format(card) for card in [self.deck.draw() for i in range(3)]])
    
    def turn(self):
        return self.deck.draw()
    
    def river(self):
        return self.deck.draw()

In [120]:
g = Game(1, 10)
g.deal()

print(g)

Player: 0
Chips: 10
Six of Hearts 
Eight of Diamonds 




In [109]:
print(g.flop())

Jack of Diamonds
Queen of Diamonds
Ten of Hearts



In [110]:
print(g.turn())

Queen of Clubs


In [111]:
print(g.river())

Five of Hearts


In [112]:
river = d.river()
print(river)

Jack of Diamonds
